# Goals
- Learn how to create 3D interpolators for plasma pprofiles to be later used by cherab and raysect
- See the logic of creating a 3D interpolator from a 1D profile

# Contents
- Usage of builtin EFITEquilibrium functions to generate interpolatots automatically
- Usage of Interpolate1DCubic, Interpolate2DCubic, IsoMapper2D, AxisymmetricMapper to create 3D interpolator step by step

In [ ]:
from cherab.core.math import Interpolate1DCubic, Interpolate2DCubic, IsoMapper2D, AxisymmetricMapper
from cherab.compass.equilibrium.equilibrium import COMPASSEquilibrium

import numpy as np
from os.path import expanduser
import matplotlib.pyplot as plt

Let's just use double parabolic profile for a plasma profile:

$Q(r) = (Q_{rmin} - Q_{rmax}) \left(1 - \left(\frac{r - r_{min}}{r_{max} - r_{min}}\right)^p\right) ^ q + Edge$

In [ ]:
def doubleparabola(r, Centre, Edge, p, q):
        return (Centre - Edge) * np.power((1 - np.power((r - r.min()) / (r.max() - r.min()), p)), q) + Edge

Generate the basic plasma profiles with normalized poloidal flux as a free variable. If the profiles do not end with 0 at the edge, effects cause by extrapolating the plasma quantity outside the interpolation region (here the confined plasma) should be watched for.

In [ ]:
psi_1d = np.linspace(0,1,100) # 1d psi coordinate to use for mapping 

#generate electon temperature profile
te_core = 1200 # electron temperature in the plasma core
te_edge = 40 # if this is not 0 extrapolation should be checked
te_p = 2 #profile shape parameter
te_q = 2 # profile shappe parameter
te_profile = doubleparabola(psi_1d, te_core, te_edge, te_p, te_q)

#now electron density profile
ne_core = 8e19 # electron temperature in the plasma core
ne_edge = 1e19 # if this is not 0 extrapolation should be checked
ne_p = 4
ne_q = 2
ne_profile = doubleparabola(psi_1d, ne_core, ne_edge, ne_p, ne_q)

In [ ]:
figprfs, ax = plt.subplots(figsize = (12,6))
ax2 = ax.twinx()
ax.plot(psi_1d, te_profile, "C0", label="Te")
ax.plot([],[], "C3", label="ne")
ax.set_xlabel("$\psi_n$")
ax.set_ylabel("Te [eV]")

ax2.plot(psi_1d, ne_profile, "C3", label="ne")
ax2.set_ylabel("ne [m$^{-3}$]")
ax.legend()
figprfs.tight_layout()

Get equilibrium slice from COMPASS EFIT equilibrium example to use for mapping. If you wonder, look into COMPASS_EFIT.ipynb

In [ ]:
path = "../data/efit_17636.h5" # path to the efit file
shot_time=1.135
equilibrium = COMPASSEquilibrium(path=path)
equilibrium_slice = equilibrium.time(shot_time)

# Generate 3D profile objects

Now we have to know how Raysect and Cherab treat things. All quantities (plasma profiles) are interpolators in a 3D cartesian space. This means we need to map our 1d profiles into 3d using a plasma equilibrium (example of compass EFIT in COMPASS_EFIT). 

## 1. Easy way using built-in functions of the EFITEquilibrium class
EFITEquilibrium.map3d() and EFITEquilibrium.map2d() functions return 3d and 2d interpolators, respectively.

In [ ]:
te = equilibrium_slice.map3d((psi_1d, te_profile))
ne = equilibrium_slice.map3d((psi_1d, ne_profile))


Now lets use the created 3D interpolators to get 2d maps to see the result

In [ ]:
xrange = np.arange(equilibrium_slice.r_data.min(), equilibrium_slice.r_data.max(), 0.01)
yrange = np.arange(equilibrium_slice.z_data.min(), equilibrium_slice.z_data.max(), 0.01)
psi_test = np.zeros((yrange.shape[0], xrange.shape[0]))
te_test = np.zeros((yrange.shape[0], xrange.shape[0]))
ne_test = np.zeros((yrange.shape[0], xrange.shape[0]))

for i, x in enumerate(xrange):
    for j, y in enumerate(yrange):
        psi_test[j, i] = equilibrium_slice.psi_normalised(x, y)
        te_test[j, i] = te(x, 0, y)
        ne_test[j, i] = ne(x, 0, y)

In [ ]:
fig_profiles, ax = plt.subplots(1, 2, figsize=(10,10))
ax[0].contourf(xrange, yrange, te_test,20)
ax[0].contour(xrange, yrange, psi_test,10, colors="w")
ax[0].set_title("te")
ax[0].set_aspect(1)
ax[0].plot(equilibrium_slice.limiter_polygon[:,0], equilibrium_slice.limiter_polygon[:,1],"-r")

ax[1].contourf(xrange, yrange, ne_test,20)
ax[1].contour(xrange, yrange, psi_test,10, colors="w")
ax[1].plot(equilibrium_slice.limiter_polygon[:,0], equilibrium_slice.limiter_polygon[:,1],"-r")
ax[1].set_title("ne")
ax[1].set_aspect(1)

##  2. Longer, harder, more boring but more instructive way
- To show the logic behind mapping and interpolating

First we need to get the ppsi normaliized 2d map and its coordinates

In [ ]:
#get the 2d poloidal psi map and r and z coordinates
psin_2d_profile = ((equilibrium_slice.psi_data - equilibrium_slice.psi_axis)/
        (equilibrium_slice.psi_lcfs - equilibrium_slice.psi_axis)) #get a psi normalized 2d map
psin_2d_r = equilibrium_slice.r_data
psin_2d_z = equilibrium_slice.z_data

psin_2d = Interpolate2DCubic(psin_2d_r, psin_2d_z, psin_2d_profile)


And then in steps from 1D to 3D:
1. interpolation of our 1d profiles using built in cherab interpolator Interpolate1DCubic
2. mapping of the interpolated profile onto the equilibrium using IsoMapper2D (Interpolate2DCubic can be used if we have 2D profiles directly)
3.  Exploiting the tokamak axisymmetry to map to 3D by using AxisymmetricMapper. The 3D objects are needed for raytracing.

Beware of the value of the paramer extrapolate=True, allowing extrapolations in the profiles. 


In [ ]:
#create 3D electron temperature plasma profiles
te_1d = Interpolate1DCubic(psi_1d, te_profile, extrapolate=True) #interpolate profile
te_2d = IsoMapper2D(equilibrium_slice.psi_normalised, te_1d) #map to 2D
te_3d = AxisymmetricMapper(te_2d) #map to 3D

#and now electron density
ne_1d = Interpolate1DCubic(psi_1d, ne_profile, extrapolate=True) #interpolate profile
ne_2d = IsoMapper2D(psin_2d, ne_1d) #map to 2D
ne_3d = AxisymmetricMapper(ne_2d) #map to 3D



Now lets check the outcomes

In [ ]:
xrange = np.arange(equilibrium_slice.r_data.min(), equilibrium_slice.r_data.max(), 0.01)
yrange = np.arange(equilibrium_slice.z_data.min(), equilibrium_slice.z_data.max(), 0.01)
psi_test = np.zeros((yrange.shape[0], xrange.shape[0]))
te_test = np.zeros((yrange.shape[0], xrange.shape[0]))
ne_test = np.zeros((yrange.shape[0], xrange.shape[0]))

for i, x in enumerate(xrange):
    for j, y in enumerate(yrange):
        psi_test[j, i] = equilibrium_slice.psi_normalised(x, y)
        te_test[j, i] = te_3d(x, 0, y)
        ne_test[j, i] = ne_3d(x, 0, y)

In [ ]:
fig_profiles, ax = plt.subplots(1, 2, figsize=(10,10))
ax[0].contourf(xrange, yrange, te_test,20)
ax[0].contour(xrange, yrange, psi_test,10, colors="w")
ax[0].set_title("te")
ax[0].set_aspect(1)
ax[0].plot(equilibrium_slice.limiter_polygon[:,0], equilibrium_slice.limiter_polygon[:,1],"-r")

ax[1].contourf(xrange, yrange, ne_test,20)
ax[1].contour(xrange, yrange, psi_test,10, colors="w")
ax[1].plot(equilibrium_slice.limiter_polygon[:,0], equilibrium_slice.limiter_polygon[:,1],"-r")
ax[1].set_title("ne")
ax[1].set_aspect(1)